In [16]:
import mysql.connector

In [17]:
# Connection parameters
host = "localhost"
user = "rasyid"
password = "yggdra01."
database = "kalbe"

# Establish the connection
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database,auth_plugin='mysql_native_password'
)


In [18]:
# Create a cursor object
cursor = conn.cursor()

In [19]:
try:
    # Specify the name of the table to be deleted
    table_name = "inventory"  # Replace with the name of the table you want to delete

    # Define the SQL command to delete the table
    delete_table_sql = f"DROP TABLE IF EXISTS {table_name}"

    # Execute the SQL command to delete the table
    cursor.execute(delete_table_sql)
    print(f"Table '{table_name}' has been deleted successfully!")

    # Commit the changes to the database
    conn.commit()

except mysql.connector.Error as err:
    print(f"Error: {err}")

Table 'inventory' has been deleted successfully!


In [5]:
data_to_insert = [
    (2341, "Promag Tablet", 3000, 100),
    (2342, "Hydro Coco 250ML", 7000, 20),
    (2343, "Nutrive Benecol 100ML", 20000, 30),
    (2344, "Blackmores Vit C 500Mg", 95000, 45),
    (2345, "Entrasol Gold 370G", 90000, 120)
]

In [6]:
# Define the SQL command to create the table
create_table_sql = """
CREATE TABLE inventory (
    ITEM_CODE INT PRIMARY KEY,
    ITEM_NAME VARCHAR(50) NOT NULL,  -- Not null constraint
    ITEM_PRICE INT DEFAULT 0,        -- Default value constraint
    ITEM_TOTAL INT CHECK (ITEM_TOTAL >= 0)  -- Check constraint
)
"""


# Execute the SQL command to create the table
cursor.execute(create_table_sql)

# Commit the transaction (save changes)
conn.commit()

In [7]:
Insert_sql = (
    "INSERT INTO INVENTORY (ITEM_CODE, ITEM_NAME, ITEM_PRICE, ITEM_TOTAL) "
    "VALUES (%s, %s, %s, %s)"
)
for i in range(len(data_to_insert)):
    cursor.execute(Insert_sql, data_to_insert[i])
conn.commit()

In [8]:
# Define the SQL query
sql_query = """
SELECT *
FROM inventory
ORDER BY ITEM_TOTAL DESC
LIMIT 1
"""

try:
    # Execute the SQL query
    cursor.execute(sql_query)

    # Fetch the result
    result = cursor.fetchall()
    

    if result:
        highest_item_name = result[0][1]
        highest_item_total = result[0][3]
        print(f"The item with the highest total is: {highest_item_name} with total {highest_item_total}")
    else:
        print("No data found in the inventory table.")

except mysql.connector.Error as err:
    print(f"Error: {err}")

The item with the highest total is: Entrasol Gold 370G with total 120


In [9]:
# Define the SQL query
sql_query = """
SELECT *
FROM inventory
"""

cursor.execute(sql_query)
result = cursor.fetchall()
print(result)

[(2341, 'Promag Tablet', 3000, 100), (2342, 'Hydro Coco 250ML', 7000, 20), (2343, 'Nutrive Benecol 100ML', 20000, 30), (2344, 'Blackmores Vit C 500Mg', 95000, 45), (2345, 'Entrasol Gold 370G', 90000, 120)]


In [10]:
try:
    # Find the ITEM_CODE associated with the item with the highest ITEM_TOTAL
    sql_query = """
    SELECT ITEM_CODE
    FROM inventory
    WHERE ITEM_TOTAL = (SELECT MAX(ITEM_TOTAL) FROM inventory)
    """

    cursor.execute(sql_query)
    result = cursor.fetchone()

    if result:
        highest_item_code = result[0]

        # Update the ITEM_PRICE for the item with the highest ITEM_TOTAL
        new_price = 120000  # Replace with the desired new price
        update_query = """
        UPDATE inventory
        SET ITEM_PRICE = %s
        WHERE ITEM_CODE = %s
        """

        cursor.execute(update_query, (new_price, highest_item_code))

        # Commit the changes to the database
        conn.commit()
        print(f"The ITEM_PRICE for ITEM_CODE {highest_item_code} has been updated to {new_price}.")

    else:
        print("No data found in the inventory table.")

except mysql.connector.Error as err:
    print(f"Error: {err}")


The ITEM_PRICE for ITEM_CODE 2345 has been updated to 120000.


In [11]:
# Define the SQL query
sql_query = """
SELECT *
FROM inventory
"""

cursor.execute(sql_query)
result = cursor.fetchall()
print(result)

[(2341, 'Promag Tablet', 3000, 100), (2342, 'Hydro Coco 250ML', 7000, 20), (2343, 'Nutrive Benecol 100ML', 20000, 30), (2344, 'Blackmores Vit C 500Mg', 95000, 45), (2345, 'Entrasol Gold 370G', 120000, 120)]


In [12]:
try:
    # Attempt to insert an item with a duplicate ITEM_CODE
    insert_query = """
    INSERT INTO inventory (ITEM_CODE, ITEM_NAME, ITEM_PRICE, ITEM_TOTAL)
    VALUES (2343, 'Duplicate Item', 100, 5)
    """

    cursor.execute(insert_query)

    # Commit the changes to the database
    conn.commit()
    print("Insertion succeeded.")

except mysql.connector.Error as err:
    # Handle the error when a duplicate primary key is detected
    print(f"Error: {err}")

Error: 1062 (23000): Duplicate entry '2343' for key 'inventory.PRIMARY'


In [13]:
try:
    # Find the ITEM_CODE associated with the item with the lowest ITEM_TOTAL
    min_total_query = """
    SELECT ITEM_CODE
    FROM inventory
    WHERE ITEM_TOTAL = (SELECT MIN(ITEM_TOTAL) FROM inventory)
    """

    cursor.execute(min_total_query)
    result = cursor.fetchone()

    if result:
        lowest_item_code = result[0]

        # Delete the item with the lowest ITEM_TOTAL
        delete_query = """
        DELETE FROM inventory
        WHERE ITEM_CODE = %s
        """
        cursor.execute(delete_query, (lowest_item_code,))

        # Commit the changes to the database
        conn.commit()
        print(f"The item with ITEM_CODE {lowest_item_code} has been deleted.")

    else:
        print("No data found in the inventory table.")

except mysql.connector.Error as err:
    print(f"Error: {err}")

The item with ITEM_CODE 2342 has been deleted.


In [14]:
# Define the SQL query
sql_query = """
SELECT *
FROM inventory
"""

cursor.execute(sql_query)
result = cursor.fetchall()
print(result)

[(2341, 'Promag Tablet', 3000, 100), (2343, 'Nutrive Benecol 100ML', 20000, 30), (2344, 'Blackmores Vit C 500Mg', 95000, 45), (2345, 'Entrasol Gold 370G', 120000, 120)]


In [ ]:
#conn.commit()

In [15]:
cursor.close()
conn.close()